# __Queries__

#### __Some Questions for the datawarehouse to answer:__

1. Which program has the most professors? Most classes offered?
2. Which course has the most classes offered?
3. What term are the most classes offered?
4. Which classes leave the most seats open?

__Import Packages__

In [1]:
%load_ext sql
import pandas as pd
import sqlite3

In [2]:
%sql sqlite:///CourseDataWarehouse.db

'Connected: @CourseDataWarehouse.db'

### __1. Which program has the most professors? Most classes offered?__

- This can be used to determine which program spends the msot money on professors.
- By determining this, the school can then analyze the amount of money this program brings in in comparison to what they are giving out.
- If the school needs to downsize, they can use this data to get overall counts of departments that are more easily downsized, and look into the possibility of increasing class size and dereasing number of classes offered.

#### __1A. Which program has the most professors?__

In [3]:
%%sql
SELECT ProgramName, ProgramCode, COUNT(Distinct ProfessorID) AS ProfessorCount
FROM ClassFactTable 
    LEFT JOIN ProgramDimension USING (ProgramID)
GROUP BY ProgramCode
ORDER BY COUNT(DISTINCT ProfessorID)DESC
LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


ProgramName,ProgramCode,ProfessorCount
Nursing,NS,143
English,EN,90
Honors,HR,62
Mathematics,MA,55
Family Therapy,FT,46
None,None,43
Education,ED,42
Biology,BI,41
Religious Studies,RS,39
Economics,EC,36


- __The Nursing program has the greatest number of professors, 143, followed by English and Honors__
    - Next Steps: look into the 'None' column for ProgramName and ProfessorCount

In [4]:
%%sql
SELECT DISTINCT ProgramName, ProgramCode, ProfessorDimension.Name
FROM ClassFactTable 
    LEFT JOIN ProgramDimension USING (ProgramID)
    LEFT JOIN ProfessorDimension USING (ProfessorID)
WHERE ProgramCode like '%WS';

 * sqlite:///CourseDataWarehouse.db
Done.


ProgramName,ProgramCode,Name
"Women, Gender, and Sexuality Studies",WS,Anna M. Lawrence
"Women, Gender, and Sexuality Studies",WS,Emily J. Orlando


#### __1B. Which program has the most classes offered?__

In [5]:
%%sql
SELECT ProgramName, ProgramCode, COUNT(DISTINCT ClassID) AS ClassesOffered
FROM ClassFactTable 
    LEFT JOIN ProgramDimension USING (ProgramID)
GROUP BY ProgramCode
ORDER BY COUNT(DISTINCT ClassID)DESC
LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


ProgramName,ProgramCode,ClassesOffered
Nursing,NS,1340
English,EN,937
Biology,BI,770
Mathematics,MA,660
Psychology,PY,549
Chemistry,CH,469
Philosophy,PH,463
Accounting,AC,450
Economics,EC,422
Spanish,SP,421


- __The Nursing program also has the greatest number of classes offered, 1340, followed by English and Biology__

### __2. Which course has the most classes offered?__

- This tells us which course at Fairfield offerest the most classes.
- Going off the previous query, this can further our analysis to which classes are offered the most, and if this number can be decreased or should be increased.

In [6]:
%%sql
SELECT DISTINCT(Title), COUNT(DISTINCT ClassID) AS ClassesOffered
FROM ClassFactTable 
GROUP BY Title
ORDER BY COUNT(DISTINCT ClassID) DESC
LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


Title,ClassesOffered
Texts and Contexts II: Writing About Literature,272
Texts and Contexts I: Writing as Craft and Inquiry,258
Introduction to Philosophy,213
Origins of the Modern World Since 1500,182
First Year Experience,156
Mental Health Nursing Clinical,129
Accelerated Statistics,120
Introduction to Microeconomics,120
Pediatric Nursing Clinical,114
Introduction to Macroeconomics,102


__- Texts and Contexts II: Writing About Literature & exts and Contexts I: Writing as Craft and Inquiry have offered the msot classes over the past few years__

### __3. What term are the most courses offered?__

- By looking into the number of classes offered in comparision to the term and year, we can see how classes have either increased or decreased over the years.
- It shows the growth of the school over the years, and can help with determining if the class growth is relative to the student and profit growth.

In [7]:
%%sql
SELECT Term, COUNT(Term) AS NumberOfClasses
FROM ClassFactTable
GROUP BY Term
ORDER BY COUNT(Term) DESC
;

 * sqlite:///CourseDataWarehouse.db
Done.


Term,NumberOfClasses
Spring2019,2749
Spring2018,2562
Spring2017,2550
Spring2015,2540
Spring2016,2476
Fall2018,2340
Fall2016,2268
Fall2014,2263
Fall2017,2240
Fall2015,2217


### __4. Which classes leave the most seats open?__

- This can help analyze whether the number of classes being offered needs to be decresed due to the classes not being filled.
- There are many classes that are over filled with students on wait lists, and there are also some with only a few students registered.
- Can the classes that are not getting filled have less offerings in order for the courses that are filled to have more offerings?

In [8]:
%%sql
SELECT ProgramCode, Title, Capacity, Actual, Remaining
FROM ClassFactTable
    JOIN ProgramDimension USING (ProgramID)
WHERE Actual < Remaining
GROUP BY Title
ORDER BY Remaining DESC
LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


ProgramCode,Title,Capacity,Actual,Remaining
EC,Engineering Applications of Numerical Methods,65,20,45
TA,Theatre Fairfield: Performance Workshop,50,8,42
TA,Theatre Fairfield: Performance Practicum,50,9,41
MU,Instrumental Ensembles,50,13,37
MU,University Symphony Orchestra,50,16,34
MA,"Calculus II for Chemistry, Engineering, and Physics Majors",30,5,25
CH,General Chemistry II,35,11,24
ME,Heat and Mass Transfer,25,1,24
AY,Special Topics: North American Indians,28,4,24
ME,Vibration Analysis,25,1,24


In [9]:
%%sql
vacuum;

 * sqlite:///CourseDataWarehouse.db
Done.


[]